In [1]:
import numpy as np

In [2]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/MOIN_DA_PROJECTS/PYTHON_IMDB_Project/

In [ ]:
%pwd

In [ ]:
%ls

In [7]:
df = pd.read_csv('IMDB_data_set.csv')

In [12]:
df.head()

,ID,Title,Genre,Director,Year,Runtime_minutes,Rating,Votes,Revenue_millions
0,1,Guardians of the Galaxy,Action,James Gunn,2014,121,8.1,757074,333.13
1,2,Prometheus,Adventure,Ridley Scott,2012,124,7.0,485820,126.46
2,3,Split,Horror,M. Night Shyamalan,2016,117,7.3,157606,138.12
3,4,Sing,Animation,Christophe Lourdelet,2016,108,7.2,60545,270.32
4,5,Suicide Squad,Action,David Ayer,2016,123,6.2,393727,325.02


In [11]:
df.shape

(1000, 9)

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [19]:
df.isna().sum()

ID                    0
Title                 0
Genre                 0
Director              0
Year                  0
Runtime_minutes       0
Rating                0
Votes                 0
Revenue_millions    128
dtype: int64

In [24]:
df.dropna(how = "any", inplace = True)
df.shape

(872, 9)

Data slicing:

We need to slice and dice the data to get a fair understanding of top earning movies, underrated movies and some additional statistics such as yearly revenue and average ratings.

How many movies have revenue higher than 75 million?

In [30]:
df[df["Revenue_millions"] > 75]["Title"].count()


318

How many movies have revenue greater than 50 million but rating less than 7?

In [36]:
df[(df["Revenue_millions"] > 50) & (df["Rating"] <7)]["Title"].count()

211

What is the total revenue generated by movies in the year 2015?

In [40]:
df[df["Year"] == 2015]["Revenue_millions"].sum()

8854.119999999999

What is the average rating for the genre adventure in the year 2015?

In [47]:
df[(df["Genre"] == "Adventure") & (df["Year"] == 2015)]["Rating"].mean()

6.8

Which year generated the highest revenue?

In [75]:
df_1 = df.groupby( by = "Year")["Revenue_millions"].sum().sort_values(ascending = False)
df_1


Year
2016    11211.65
2015     8854.12
2014     7997.40
2013     7666.72
2012     6910.29
2010     5989.65
2011     5431.96
2009     5292.26
2008     5053.22
2007     4306.23
2006     3624.46
Name: Revenue_millions, dtype: float64

Data analysis:

Now that we have explored and understood the data, we want to create certain reports from the IMDB dataset. These reports will help the stakeholders analyze which type of movie makes more money. How does the rating of the movie affect the revenue of the movie?



Create a genre-level report with metrics average rating, the average number of votes, and average revenue.

In [136]:
df_2 = round(df.groupby(by = 'Genre')["Rating", "Votes", "Revenue_millions"].mean(),2)
df_2.sort_values(by = ["Revenue_millions"], ascending = False)



<ipython-input-136-4039033eb6d3>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_2 = round(df.groupby(by = 'Genre')["Rating", "Votes", "Revenue_millions"].mean(),2)


,Rating,Votes,Revenue_millions
Genre,,,
Animation,7.31,216865.15,191.22
Action,6.64,246573.64,122.09
Adventure,6.97,238187.01,113.45
Mystery,6.91,256687.00,64.38
Fantasy,6.65,177093.00,63.11
Romance,7.10,208632.00,62.45
Biography,7.38,162391.95,55.95
Comedy,6.55,127876.69,51.58
Sci-Fi,4.50,85587.00,42.64


- Create a report to showcase the revenue of each movie, as % revenue concerning the total revenue of the respective genre and year of the movie. For example if a movie ‘ABC’ has genre ‘Action’ and released in 2015, then % revenue will be - (Revenue of the movie ‘ABC’ *100)/ (Total revenue of the genre ‘Action’ in 2015) What is the  % revenue of the movie ‘Split’ in its respective genre and year?

In [ ]:
df_revenue_per_year = df.groupby(by = ["Year", "Genre"]).agg({"Revenue_millions" : "sum"})
df_revenue_per_year

In [ ]:
df_revenue_per_year.rename(columns = {"Revenue_millions" : "Total_revenue"}, inplace = True)
df_revenue_per_year

In [ ]:
df_main = pd.merge(df, df_revenue_per_year, left_on = ["Genre", "Year"], right_on = ["Genre", "Year"], how = "inner")
df_main

In [110]:
df_main["%revenue"] = df_main.apply(lambda x : x["Revenue_millions"]*100/x["Total_revenue"], axis = 1)
df_main[df_main["Title"].isin(["Split"])]

,ID,Title,Genre,Director,Year,Runtime_minutes,Rating,Votes,Revenue_millions,Total_revenue,%revenue
40,3,Split,Horror,M. Night Shyamalan,2016,117,7.3,157606,138.12,469.47,29.42041


How many directors have created movies in the highest number of genres?

In [134]:
df_dir = df.groupby(by = "Genre")["Director"].count()
df_dir

Genre
Action       276
Adventure     71
Animation     47
Biography     60
Comedy       156
Crime         54
Drama        156
Fantasy        2
Horror        35
Mystery       11
Romance        1
Sci-Fi         2
Thriller       1
Name: Director, dtype: int64

In [ ]:
df_dir_1 = df.loc[df["Genre"] == "Action", :]
df_dir_1

In [124]:
df_dir_1["Director"].nunique()

185